In [ ]:
import glob
import pandas as pd
import re
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
from matplotlib import rcParams

rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Data_holder():
    def __init__(self, stock_prices_raw, train_val_divided, val_flag):
        self.stock_close_price_index = stock_prices_raw.loc[:,'open':'volume'].columns.get_loc('close')
        self.stock_prices = stock_prices_raw.loc[:,'open':'volume'].values
        self.train_val_divided = train_val_divided
        self.val_flag = val_flag
        if self.val_flag != True:
            self.stock_prices = self.stock_prices[:int(self.stock_prices.shape[0]*self.train_val_divided)]
        if self.val_flag == True:
            self.stock_prices = self.stock_prices[int(self.stock_prices.shape[0]*self.train_val_divided):]


    @staticmethod
    def prepare_data(file_source):
        """株価データのcsvファイルをpandas.core.frame.DataFrame形式でloadする関数。

        Args:
            str: csvファイルのuri。

        Return:
            pandas.core.frame.DataFrame: 株価データ。「列名：日付,始値,高値,低値,終値,出来高,取引総額」、「行名：日付」の表。

        """
        stock_data = pd.read_csv(file_source, engine = "python")
        stock_data = stock_data.set_index('date')
        temp = file_source.replace(storage_uri, '')
        temp = re.sub(r'YYYYY', '', temp)
        latest_data = re.sub(r'.csv', '', temp)
        latest_data = [float(x.strip()) for x in latest_data.split(',')]
        temp = pd.DataFrame([latest_data[1:]], columns=stock_data.columns, index=[int(latest_data[0])], dtype=float)
        stock_data = stock_data.append(temp,ignore_index=False)
        return stock_data


    def fragment_normalize_data(self, window_size, predicted_day_num):
        fragment_num = self.stock_prices.shape[0] - window_size - predicted_day_num
        stock_price_flagments = np.zeros((fragment_num, window_size+predicted_day_num, feature_num))
        XXXXX = YYYYY #test時に出力を株価に変換するために必要。

        #正規化。
        """
        詳細は割愛。
        """

        stock_price_labels = torch.tensor(stock_price_flagments[:, window_size+predicted_day_num-1, self.stock_close_price_index], dtype=torch.float, device=device).view(-1, 1)
        stock_price_flagments = torch.tensor(stock_price_flagments[:, :window_size, :], dtype=torch.float, device=device)

        if self.val_flag != True:
            return stock_price_labels, stock_price_flagments
        if self.val_flag == True:
            return stock_price_labels, stock_price_flagments, XXXXX


class StockDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, phase):
        self.sequence = x
        self.label = y
        self.phase = phase


    def __len__(self):
        return len(self.label)


    def __getitem__(self, idx):
        out_data = self.sequence[idx]
        out_label =  self.label[idx]
        return out_data, out_label


class LSTMC_full_connected_layer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
            super().__init__()
            self.input_size = input_size
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            self.batch_size = batch_size
            self.lstm = nn.LSTM(input_size=self.input_size,
                                hidden_size=self.hidden_size,
                                num_layers=self.num_layers,
                                batch_first=True
                                )
            self.dense = nn.Linear(hidden_size, 1)


    def forward(self, x_data, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, hidden_layer_size)
        c0 = torch.zeros(self.num_layers, batch_size, hidden_layer_size)

        #隠れ層、cellの内部状態は使用しない。
        lstm_out, (hn, cn) = self.lstm(x_data, (h0, c0))
        linear_out = self.dense(lstm_out[:,-1,:].view(x_data.size(0), -1))
        return torch.sigmoid(linear_out)


def loss_func(pred_value, true_value):
    small_value = 1e-4
    pred_value_flattened = pred_value.view(-1)
    true_value_flattened = true_value.view(-1)
    squared_diff_sum = torch.sum((pred_value_flattened - true_value_flattened)**2)
    loss = (squared_diff_sum + small_value)/(pred_value_flattened.size(0) + small_value)
    return loss


def train_model(dataloaders_dict):
    model = LSTMC_full_connected_layer(feature_num, hidden_layer_size, num_layers)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_train = np.zeros((epochs, 2))
    loss_val = np.zeros((int(epochs/5), 2))

    for i in range(epochs):
        total_loss = 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                #optimizer.zero_grad() （注1-1）2020/12/16 コメントアウト。詳しくは下記（注1）を参照のこと。
                print('（train）')
            else:
                if((i+1) % 5 == 0):
                    model.eval()
                    print('-------------')
                    print('（val）')
                else:
                    continue

            count = 0
            for x, y in dataloaders_dict[phase]:
                count += 1
                with torch.set_grad_enabled(phase == 'train'):
                    y_pred = model(x, batch_size)
                    loss = loss_func(y_pred, y)
                    total_loss += loss.item()
                    if (phase == 'train'):
                        optimizer.zero_grad() #（注1-2）2020/12/16 追加。詳しくは下記（注1）を参照のこと。
                        loss.backward()
                        optimizer.step()

            total_loss /= count
            if phase == 'train':
                loss_train[i, 0] = i+1
                loss_train[i, 1] = total_loss
            if phase == 'val' and (i+1) % 5 == 0:
                loss_val[int((i+1)/5-1), 0] = i+1
                loss_val[int((i+1)/5)-1, 1] = total_loss
            print(f'epoch: {i+1:3} loss: {total_loss:10.8f}')
    return model, loss_train, loss_val


storage_uri = YYYYY
gl = glob.glob(YYYYY)

window_size = Y
feature_num = 5
train_val_divided = 0.7

batch_size = Y
hidden_layer_size = Y
num_layers = 1
epochs = 50


def train_and_test_NN(stock_prices_raw1, stock_prices_raw2, predicted_day_num):
    data_holder_t = Data_holder(stock_prices_raw1, train_val_divided, val_flag=False)
    data_holder_v = Data_holder(stock_prices_raw2, train_val_divided, val_flag=True)

    stock_price_labels_t, stock_price_flagments_t = data_holder_t.fragment_normalize_data(window_size, predicted_day_num)
    stock_price_labels_v, stock_price_flagments_v, XXXXX = data_holder_v.fragment_normalize_data(window_size, predicted_day_num)

    train_dataset = StockDataset(stock_price_flagments_t, stock_price_labels_t, 'train')
    val_dataset = StockDataset(stock_price_flagments_v, stock_price_labels_v, 'val')

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

    model, loss_train, loss_val = train_model(dataloaders_dict)
    test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=True)
    prices_pred = np.zeros((len(test_dataloader)))
    prices_true = np.zeros((len(test_dataloader)))

    count = 0
    for x, y in test_dataloader:
        y_pred = model(x, batch_size=1).item()*(YYYYY)
        y = y.item()*(YYYYY)
        prices_pred[count] = y_pred
        prices_true[count] = y
        count += 1
    return prices_pred, prices_true, loss_train, loss_val


stock_prices_raw = Data_holder.prepare_data(gl[0])
drawing_timing = [1, 5, 20, 60, 120]
predicted_day_range = 120
mean_absolute_errors = np.zeros((predicted_day_range))
mean_absolute_price_changing_rate = np.zeros((predicted_day_range))
fig0, ax0 = plt.subplots()

for i in range(predicted_day_range):
    predicted_day_num = i+1
    prices_pred, prices_true, loss_train, loss_val = train_and_test_NN(stock_prices_raw, stock_prices_raw, predicted_day_num)

    mean_absolute_errors[i] = mean_absolute_error(prices_pred/prices_true, prices_true/prices_true)*100

    tmp = 0
    for j in range(len(prices_true)-predicted_day_num):
        tmp += abs((prices_true[j+predicted_day_num] - prices_true[j]) / prices_true[j])
    mean_absolute_price_changing_rate[i] = tmp / (len(prices_true)-predicted_day_num)*100

    if predicted_day_num in drawing_timing:
        x = np.linspace(predicted_day_num-1, len(prices_true)+predicted_day_num-1, len(prices_pred))
        if predicted_day_num == 1:
            ax0.plot(x, prices_true, label='真値', color='black')
        ax0.plot(x, prices_pred, label=str(predicted_day_num)+'営業日後予測')
        ax0.set_xlabel('営業日')
        ax0.set_ylabel('株価（円）')
        ax0.legend()
        ax0.xaxis.set_minor_locator(tick.MultipleLocator(1))
        ax0.grid(which='minor', linewidth=0.5)
        fig0.savefig('日経平均株価予測.png', format="png", dpi=500)


        fig1, ax1 = plt.subplots()
        ax1.plot(loss_train[:, 0], loss_train[:, 1], label='損失関数（訓練）')
        ax1.plot(loss_val[:, 0], loss_val[:, 1], label='損失関数（検証）')
        ax1.set_xlabel('エポック数')
        ax1.set_title('損失関数（'+str(predicted_day_num)+'営業日後予測）')
        ax1.set_ylim(0.0, 0.2)
        ax1.legend()
        plt.savefig('損失関数'+str(predicted_day_num)+'.png', format="png", dpi=500)


fig = plt.figure()
x = np.linspace(0, predicted_day_range-1, predicted_day_range)
plt.plot(x, mean_absolute_price_changing_rate, label='株価平均絶対変化率', color='black')
plt.plot(x, mean_absolute_errors, label='予測値平均絶対誤差', color='red')
plt.xlabel('n営業日後予測')
plt.ylabel('平均絶対誤差・株価平均絶対変化率（%）')
plt.legend()
plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
plt.grid(which='minor', linewidth=0.5)
plt.savefig('平均絶対誤差・株価平均絶対変化率（%）.png', format="png", dpi=500)
 

（注1）（注1-1）のoptimizer.zero_grad()をコメントアウトし、（注1-2）にoptimizer.zero_grad()を追加した（2020年12月16日）。（注1-1）の位置にoptimizer.zero_grad()があると、epochが終わるまで損失関数の勾配が初期化されない。学習はミニバッチ勾配降下法で行われるので、bachごとに勾配は初期化されなければならない。そのため、optimizer.zero_grad()の正しい位置は、（注1-1）ではなく、（注1-2）。この文章内にあるグラフは、optimizer.zero_grad()が（注1-1）の位置にあるプログラムから得られたもの。

